# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn import tree

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
def num_var(x,y,z,i,j):
    temp = df[df[x].notnull()]
    dtree = tree.DecisionTreeClassifier(random_state=42, max_depth=i, min_samples_leaf=j)
    dtree = dtree.fit(temp[[x]],temp[y])
    temp[z] = dtree.apply(temp[[x]])
    gb = temp.groupby(z).agg({'not.fully.paid':'count', x:['min','max'], y:'mean'}).reset_index()

    temp = df[df[x].isnull()]
    temp[z] = 'MISSING'
    temp = temp.groupby(z).agg({'not.fully.paid':'count', y:'mean'}).reset_index()
    gb.columns = [z, 'count', 'min', 'max', 'Y']
    temp.columns = [z, 'count', 'Y']

    gb = pd.concat([gb, temp], axis=0, ignore_index=True)
    display(gb)

In [4]:
def cat_var(x,y):
    df[x] = df[x].fillna('MISSING')
    gb = df.groupby(x).agg({'not.fully.paid':'count',y:'mean'}).reset_index()
    display(gb)

# 2. Import datasets

In [5]:
df = pd.read_csv("../inputs/input.csv")
df['Def'] = np.where(df['not.fully.paid']=='Paid', 0, 1)
df.shape

(9578, 15)

# 3. Categorical

In [6]:
df['purpose_bin'] = np.where(df['purpose'].isin(['major_purchase','credit_card']), 0.115, 
                             np.where(df['purpose'].isin(['small_business']), 0.278, 0.161))
cat_var('purpose_bin','Def')

,purpose_bin,not.fully.paid,Def
0,0.115,1699,0.114773
1,0.161,7260,0.160606
2,0.278,619,0.277868


# 4. Numerical

In [7]:
df['credit_policy_bin'] = np.where(df['credit.policy']==0, 0.278, 0.132)
cat_var('credit_policy_bin','Def')

df['int_rate_bin'] = np.where(df['int.rate']<=0.0932, 0.056, 
                              np.where(df['int.rate']<=0.1392, 0.159, 
                                       np.where(df['int.rate']<=0.1635, 0.213, 0.301)))
cat_var('int_rate_bin','Def')

df['log_annual_inc_bin'] = np.where(df['log.annual.inc']<=9.798, 0.240, 
                                    np.where(df['log.annual.inc']<=12.092, 0.155, 0.240))
cat_var('log_annual_inc_bin','Def')

df['installment_bin'] = np.where(df['installment']<=423.77, 0.158, 
                                    np.where(df['installment']<=476.58, 0.105, 
                                             np.where(df['installment']<=829.97, 0.158, 0.290)))
cat_var('installment_bin','Def')

df['fico_bin'] = np.where(df['fico']<=657, 0.309, 
                          np.where(df['fico']<=672, 0.222, 
                                   np.where(df['fico']<=737, 0.165, 0.081)))
cat_var('fico_bin','Def')

df['days_with_cr_line_bin'] = np.where(df['days.with.cr.line']<=1110, 0.270, 0.157)
cat_var('days_with_cr_line_bin','Def')

df['revol_bal_bin'] = np.where(df['revol.bal']<=75723, 0.157, 0.256)
cat_var('revol_bal_bin','Def')

df['revol_util_bin'] = np.where(df['revol.util']<=96.5, 0.157, 0.259)
cat_var('revol_util_bin','Def')

df['inq_last_6mths_bin'] = np.where(df['inq.last.6mths']<=0, 0.117, 
                                    np.where(df['inq.last.6mths']<=2, 0.148, 
                                             np.where(df['inq.last.6mths']<=4, 0.218, 
                                                      np.where(df['inq.last.6mths']<=5, 0.273, 0.351))))
cat_var('inq_last_6mths_bin','Def')

df['delinq_2yrs_bin'] = np.where(df['delinq.2yrs']==0, 0.158, 0.175)
cat_var('delinq_2yrs_bin','Def')

df['pub_rec_bin'] = np.where(df['pub.rec']==0, 0.155, 0.247)
cat_var('pub_rec_bin','Def')

,credit_policy_bin,not.fully.paid,Def
0,0.132,7710,0.131518
1,0.278,1868,0.277837


,int_rate_bin,not.fully.paid,Def
0,0.056,1676,0.056086
1,0.159,5477,0.159394
2,0.213,1864,0.212983
3,0.301,561,0.301248


,log_annual_inc_bin,not.fully.paid,Def
0,0.155,9000,0.154889
1,0.240,578,0.240484


,installment_bin,not.fully.paid,Def
0,0.105,344,0.104651
1,0.158,8972,0.158382
2,0.290,262,0.290076


,fico_bin,not.fully.paid,Def
0,0.081,2230,0.081166
1,0.165,5612,0.164647
2,0.222,1247,0.222133
3,0.309,489,0.308793


...

,revol_bal_bin,not.fully.paid,Def
0,0.157,9281,0.156987
1,0.256,297,0.255892


,revol_util_bin,not.fully.paid,Def
0,0.157,9304,0.157137
1,0.259,274,0.259124


,inq_last_6mths_bin,not.fully.paid,Def
0,0.117,3637,0.117404
1,0.148,3846,0.148206
2,0.218,1339,0.218073
3,0.273,278,0.273381
4,0.351,478,0.351464


,delinq_2yrs_bin,not.fully.paid,Def
0,0.158,8458,0.158075
1,0.175,1120,0.175000


,pub_rec_bin,not.fully.paid,Def
0,0.155,9019,0.154673
1,0.247,559,0.246869


# 5. Export dataset

In [8]:
df = df[['Def', 'purpose_bin', 'credit_policy_bin', 'int_rate_bin', 'log_annual_inc_bin', 'installment_bin', 'fico_bin',
         'days_with_cr_line_bin', 'revol_bal_bin', 'revol_util_bin', 'inq_last_6mths_bin', 'delinq_2yrs_bin', 'pub_rec_bin']]
df.to_csv('../inputs/1_Bins.csv', index=False)
df.shape

(9578, 13)